In [1]:
#importing the libraries
import requests 
from bs4 import BeautifulSoup as BS
import re
import pandas as pd
import numpy as np

In [2]:
#generating all webpages of each county
x = ['https://www.redfin.com/county/303/CA/Alameda-County',
    'https://www.redfin.com/county/340/CA/San-Francisco-County','https://www.redfin.com/county/341/CA/San-Joaquin-County',
    'https://www.redfin.com/county/343/CA/San-Mateo-County',
    'https://www.redfin.com/county/345/CA/Santa-Clara-County']
county_links =[]
for c_l in x:
    c_text = requests.get(c_l,headers={'User-Agent': 'Mozilla/5.0'}).text
    c_soup = BS(c_text,'html5lib')
    page_number=int(c_soup.find('span',attrs={'class':'pageText'}).text.split('of ')[1])
    county_links.append(c_l)
    if page_number>1:
        for num in range(2,page_number+1):
            str_num=str(num)
            new_c_l = c_l+'/page-'+str_num
            county_links.append(new_c_l)
county_links

['https://www.redfin.com/county/303/CA/Alameda-County',
 'https://www.redfin.com/county/303/CA/Alameda-County/page-2',
 'https://www.redfin.com/county/303/CA/Alameda-County/page-3',
 'https://www.redfin.com/county/303/CA/Alameda-County/page-4',
 'https://www.redfin.com/county/303/CA/Alameda-County/page-5',
 'https://www.redfin.com/county/303/CA/Alameda-County/page-6',
 'https://www.redfin.com/county/303/CA/Alameda-County/page-7',
 'https://www.redfin.com/county/303/CA/Alameda-County/page-8',
 'https://www.redfin.com/county/303/CA/Alameda-County/page-9',
 'https://www.redfin.com/county/340/CA/San-Francisco-County',
 'https://www.redfin.com/county/340/CA/San-Francisco-County/page-2',
 'https://www.redfin.com/county/340/CA/San-Francisco-County/page-3',
 'https://www.redfin.com/county/340/CA/San-Francisco-County/page-4',
 'https://www.redfin.com/county/340/CA/San-Francisco-County/page-5',
 'https://www.redfin.com/county/340/CA/San-Francisco-County/page-6',
 'https://www.redfin.com/county/3

In [3]:
#Creating a dictionary with key - county , value - List of properties
#extracting all links in a webpage using a href tags and using regex to identify the links for properties
properties={}

for url in county_links:
    county_url = url
    county_name_list = re.findall('CA/([^/]+)',county_url)
    text = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}).text
    soup = BS(text,'html5lib')
    all_links = [a['href'] for a in soup('a') if a.has_attr('href')] #extracting all links on the webpage
    regex = r"^/CA" #extracting only links for properties listed
    one_county_prop_url = [link for link in all_links if re.match(regex,link)]
    for a in range(0, len(one_county_prop_url)):
        one_county_prop_url[a]='https://www.redfin.com'+one_county_prop_url[a]
    final_url =[]
    for l in one_county_prop_url:
        if l not in final_url:
            final_url.append(l)
    for x in county_name_list:
        if x in properties:
            properties[x]=properties[x]+final_url
        else:
            properties[x]=final_url
        
properties

ChunkedEncodingError: ("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read))

In [ ]:
#creating lists for the attributes and extracting property information using tags like div, span, etc
#using try & except blocks to identify errors and handle them

Street= []
City = []
State = []
Zipcode = []
Price = []
Beds = []
Bath = []
Sqft = []
Property_Type = []
Year_Built = []
Status = []
Walk_Score = []
Bike_Score = []
Acre=[]
count=0
for county in properties:
    for prop in properties[county]:
        print(count)
        count=count+1
        al_url = prop
        al_text = requests.get(al_url,headers={'User-Agent': 'Mozilla/5.0'}).text
        al_soup = BS(al_text,'html5lib')
        
        try:
            street = al_soup.find('div',attrs={'class':'street-address'}).text.split(',')[0]
        except:
            street=np.nan
            print(al_url)
            print('street')
        Street.append(street)
        
        try:
            city = al_soup.find('div',attrs={'class':'dp-subtext'}).text.split(', ')[0]
        except:
            city=np.nan
            print(al_url)
            print('city')
        City.append(city)
        
        try:
            state = al_soup.find('div',attrs={'class':'dp-subtext'}).text.split(', ')[1].split(' ')[0]
        except:
            state=np.nan
            print(al_url)
            print('state')
        State.append(state)
        
        try:
            zipcode = al_soup.find('div',attrs={'class':'dp-subtext'}).text.split(', ')[-1].split(' ')[-1]
            
        except:
            zipcode=np.nan
            print(al_url)
            print('zipcode')
        Zipcode.append(zipcode)    
        
        try:
            price = al_soup.find('div',attrs={'class':'statsValue'}).text.split('$')[1]
        except:
            price=np.nan
            print(al_url)
            print('price')
        Price.append(price)
        
        try:
            bed_str = al_soup.find('div',attrs={'data-rf-test-id':'abp-beds'}).text.split('B')[0]
            beds = int(bed_str)
        except:
            beds=np.nan
            print(al_url)
            print('beds')
        Beds.append(beds)
        
        try:
            bath_str =al_soup.find('div',attrs ={'data-rf-test-id':'abp-baths'}).text.split('B')[0]
            bath = float(bath_str)
        except:
            bath=np.nan
            print(al_url)
            print('bath')
        Bath.append(bath)
        
        try:
            sqft_str =al_soup.find('div',attrs ={'data-rf-test-id':'abp-sqFt'}).text.split('S')[0].replace(',','')
            sqft = float(sqft_str)
        except:
            sqft=np.nan
            print(al_url)
            print('sqft')
        Sqft.append(sqft)
        
        try:
            acre_str =al_soup.find('div',attrs ={'data-rf-test-id':'abp-sqFt'}).text.split('A')[0]
            acre = float(acre_str)
        except:
            acre=np.nan
            print(al_url)
            print('acre')
        Acre.append(acre)
        
        
        try:
            div_prop = al_soup.findAll('div',attrs={'class':'keyDetail font-weight-roman font-size-base'})
            property_type =''
            for tag in div_prop:
                if(tag.span.text=='Property Type'):
                    property_type = property_type+tag.find('span',attrs={'class':'content text-right'}).text
        except:
            property_type = np.nan
            print(al_url)
            print('prop_type')
        Property_Type.append(property_type)
        
        try:
            div_year_built = al_soup.findAll('div',attrs={'class':'keyDetail font-weight-roman font-size-base'})
            year_built =''
            for tag in div_year_built:
                if(tag.span.text=='Year Built'):
                    year_built = int(year_built+tag.find('span',attrs={'class':'content text-right'}).text)
        except:
            year_built=np.nan
            print(al_url)
            print('year')
        Year_Built.append(year_built)
        
        try:
            div_status = al_soup.findAll('div',attrs={'class':'keyDetail font-weight-roman font-size-base'})
            status =''
            for tag in div_status:
                if(tag.span.text=='Status'):
                    status = tag.find('span',attrs={'class':'DefinitionFlyoutLink inline-block underline clickable'})
                    if status == None:
                        status = tag.find('span',attrs={'class':'content text-right'})
        except:
            status=np.nan
            print(al_url)
            print('status')
        Status.append(status.text)

        try:
            walk_score = int(al_soup.find('div',attrs={'class':'transport-icon-and-percentage walkscore'}).text.split(' /')[0])
        except:
            walk_score=np.nan
            print(al_url)
            print('walk')
        Walk_Score.append(walk_score)
        
        try:
            bike_score = int(al_soup.find('div',attrs={'class':'transport-icon-and-percentage bikescore'}).text.split(' /')[0])
        except:
            bike_score=np.nan
            print(al_url)
            print('bike')
        Bike_Score.append(bike_score)
       


0
https://www.redfin.com/CA/Oakland/655-12th-St-94607/unit-112/home/1553948
acre
1
https://www.redfin.com/CA/Oakland/4475-Redding-St-94619/home/561724
acre
2
https://www.redfin.com/CA/Oakland/6501-San-Pablo-Ave-94608/unit-404/home/12116522
acre
3
https://www.redfin.com/CA/Fremont/36088-Soapberry-Cmn-94536/unit-31/home/22806783
acre
4
https://www.redfin.com/CA/Union-City/32824-Oakdale-Ct-94587/home/1877537
acre
5
https://www.redfin.com/CA/Dublin/Undisclosed-address-94568/home/1168755
acre
6
https://www.redfin.com/CA/Oakland/2580-Pleasant-St-94602/home/1727436
acre
7
https://www.redfin.com/CA/Pleasanton/1858-Brooktree-Way-94566/home/1194282
acre
8
https://www.redfin.com/CA/Alameda/215-Kevington-Pl-94502/home/1905334
acre
9
https://www.redfin.com/CA/Berkeley/1405-Harmon-St-94702/home/1817934
acre
10
https://www.redfin.com/CA/Oakland/3867-Brown-Ave-94619/home/528574
acre
11
https://www.redfin.com/CA/Hayward/27701-Eucalyptus-Ct-94544/home/1735626
acre
12
https://www.redfin.com/CA/Newark/814

In [ ]:
County=[]
for key in properties:
    for value in properties[key]:
       County.append(key)
County

In [ ]:
#creating data frame 

In [ ]:
df=pd.DataFrame(data={'Street':Street,'City':City,'County':County,'State':State,'Zipcode':Zipcode,
                      'Price':Price,
                      'WalkScore':Walk_Score,
                     'Bike_Score':Bike_Score,'Beds':Beds,'Bath':Bath,'Sqft':Sqft,'Acre':Acre,
                      'Property_Type':Property_Type,
                     'Year_Built':Year_Built,'Status':Status})

In [ ]:
df

In [ ]:
df.isna().sum()

In [ ]:
#Data cleaning

In [ ]:
(df['Beds'].isna() & df['Bath'].isna()).sum()

In [ ]:
#If both beds and Bath have null values -> that indicates that the property is an empty land and there is no
#built house. We will drop such rows

In [ ]:
df = df.dropna(subset=['Beds','Bath'],how='all')

In [ ]:
#Street, City, State, Zipcode, Price have small number of null values. We will remove such rows since that will affect
#our analysis and insights

In [ ]:
df=df.dropna(subset=['Street','City','State','Zipcode','Price'])

In [ ]:
#replacing walk score and bike score null values with score of 0. Does not make sense to delete the entire row
#based on these scores as null value indicates score of 0.

In [ ]:
df['WalkScore'].fillna(0,inplace=True)
df['Bike_Score'].fillna(0,inplace=True)

In [ ]:
#We have removed properties where both beds and baths are 0
#To deal with remaining null values in Individual columns of Beds & Baths, we will replace it with 0 since 
#that means there is either no bedroom or not bathroom in that property

In [ ]:
df['Beds'].fillna(0,inplace=True)
df['Bath'].fillna(0,inplace=True)

In [ ]:
df.isna().sum()

In [ ]:
#deleting rows where sqft is not provided(null) since sqft is a part of our analysis

In [ ]:
df=df.dropna(subset=['Sqft'])

In [ ]:
#dropping the acre column. It was only used to identify the values where we do not have sqft

In [ ]:
df=df.drop(columns=['Acre'])

In [ ]:
df=df.reset_index()

In [ ]:
df=df.drop(columns=['index'])

In [ ]:
df

In [ ]:
#remove +sign & special characters
for i in range(0,len(df)):
    if(df['Price'][i][-1]=='+'):
        df['Price'][i]=df['Price'][i][0:-1]  
    else:
        df['Price'][i]=df['Price'][i]
        

In [ ]:
#convert data type to relevant types

In [ ]:
df['Price'] = df['Price'].str.replace(',','')

In [ ]:
df['Price'] = df['Price'].astype(float)
df.loc[df['Year_Built'] == '', 'Year_Built'] = 0



In [ ]:
df['Year_Built'] = df['Year_Built'].astype(int)

In [ ]:
df.loc[df['Year_Built'] == 0, 'Year_Built'] = np.nan

In [ ]:
df.dtypes

In [ ]:
#Normalization

In [ ]:
df['Street']=df['Street'].apply(lambda x:x[0]+x[1:].lower())

In [ ]:
df['City']=df['City'].apply(lambda x:x[0]+x[1:].lower())

In [ ]:
df['State']=df['State'].apply(lambda x:x[0]+x[1:].lower())

In [ ]:
df['Property_Type']=df['Property_Type'].apply(lambda x:x[0]+x[1:].lower())

In [ ]:
df['Status']=df['Status'].apply(lambda x:x[0]+x[1:].lower())

In [ ]:
df

In [ ]:
#removing duplicates

In [ ]:
df.duplicated().value_counts()

In [ ]:
df=df.drop_duplicates()

In [ ]:
df

In [ ]:
df.kurt(numeric_only=True)

In [ ]:
df['Price'].describe()

In [ ]:
df['Beds'].describe()

In [ ]:
df['Bath'].describe()

In [ ]:
df['Sqft'].describe()

In [ ]:
#Calculating Quartiles, IQR and removing the outliers

In [ ]:
Q1_Price = df['Price'].quantile(0.25)
Q3_Price = df['Price'].quantile(0.75)
IQR_Price = Q3_Price - Q1_Price
IQR_Price

In [ ]:
Q1_Beds = df['Beds'].quantile(0.25)
Q3_Beds = df['Beds'].quantile(0.75)
IQR_Beds = Q3_Beds - Q1_Beds
IQR_Beds

In [ ]:
Q1_Bath = df['Bath'].quantile(0.25)
Q3_Bath = df['Bath'].quantile(0.75)
IQR_Bath = Q3_Bath - Q1_Bath
IQR_Bath

In [ ]:
Q1_Sqft = df['Sqft'].quantile(0.25)
Q3_Sqft = df['Sqft'].quantile(0.75)
IQR_Sqft = Q3_Sqft - Q1_Sqft
IQR_Sqft

In [ ]:
df['Price'] = df['Price'][~((df['Price'] < (Q1_Price - 1.5 * IQR_Price)) |(df['Price'] > 
                                                                           (Q3_Price + 1.5 * IQR_Price)))]

In [ ]:
df['Beds'] = df['Beds'][~((df['Beds'] < (Q1_Beds - 1.5 * IQR_Beds)) |(df['Beds'] > 
                                                                           (Q3_Beds + 1.5 * IQR_Beds)))]

In [ ]:
df['Bath'] = df['Bath'][~((df['Bath'] < (Q1_Bath - 1.5 * IQR_Bath)) |(df['Bath'] > 
                                                                           (Q3_Bath + 1.5 * IQR_Bath)))]

In [ ]:
df['Sqft'] = df['Sqft'][~((df['Sqft'] < (Q1_Sqft - 1.5 * IQR_Sqft)) |(df['Sqft'] > 
                                                                           (Q3_Sqft + 1.5 * IQR_Sqft)))]

In [ ]:
df.isna().sum() #number of outliers removed

In [ ]:
df=df.dropna() #removing the null values. The outliers were removed and replaced by null values

In [ ]:
df

In [ ]:
#Exporting to CSV

In [ ]:
df.to_csv('DA_Grp_9_Redfin_Dataset.csv',index=False)